In [1]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
import sys
import os
sys.path.append(os.path.join('..', '..', '..' ))

from shared.cifar10_dvs_dataset import CIFAR10DVSData

data_handler = CIFAR10DVSData(save_to='../../../shared/datasets')
data_handler.check_and_download()

train_dataloader, val_dataloader, test_dataloader = data_handler.get_spike_tensor_dataset(batch_size=16, train_ratio=0.7, num_time_bins=300)

c:\Users\xsb24130\AppData\Local\miniconda3\envs\loihi-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


         Only Prophesee DVS demo will not run properly.
         Please install it from https://github.com/prophesee-ai/prophesee-automotive-dataset-toolbox
CIFAR10DVS Data handler initialized. Data will be saved to '../../../shared/datasets'
Download check complete.
Splited dataset into:
 - Training set: 7000 samples
 - Validation set: 1500 samples
 - Test set: 1500 samples


MemoryError: Unable to allocate 2.95 MiB for an array with shape (238276,) and data type [('t', '<u8'), ('x', '<u2'), ('y', '<u2'), (('on', 'p'), '?')]

In [21]:
import tonic

transform = tonic.transforms.Compose([
            tonic.transforms.NumpyAsType(int)
        ])

dataset = tonic.datasets.CIFAR10DVS(
    save_to='../../../shared/datasets',
    transform=transform
)

events, label = dataset[1]

print(events.shape, label)

(234458, 4) 0


In [4]:
events, label = next(iter(train_dataloader))

print(label)
print(events.shape)

tensor([4])
torch.Size([1, 147802, 4])


In [21]:
import lava.lib.dl.slayer as slayer
import torch.nn as nn
import matplotlib.pyplot as plt
import h5py

class CNNSlayer(torch.nn.Module):
    def __init__(self):
        super(CNNSlayer, self).__init__()

        self.neuron_params = {
                'threshold'     : 1.0,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 3,
                'requires_grad' : True,     
            }
        
        # Define sequential blocks with ModuleList
        self.blocks = nn.ModuleList([
            slayer.block.cuba.Conv(self.neuron_params, 1, 32, kernel_size=3, stride=2, padding=1, weight_norm=True),
            slayer.block.cuba.Pool(self.neuron_params, kernel_size=2, stride=2, weight_norm=True),
            
            slayer.block.cuba.Conv(self.neuron_params, 32, 64, kernel_size=3, stride=2, padding=1, weight_norm=True),
            slayer.block.cuba.Pool(self.neuron_params, kernel_size=2, stride=2, weight_norm=True),
        
            slayer.block.cuba.Conv(self.neuron_params, 64, 128, kernel_size=3, stride=1, padding=1, weight_norm=True),
            slayer.block.cuba.Pool(self.neuron_params, kernel_size=2, stride=2, weight_norm=True),

            slayer.block.cuba.Dense(self.neuron_params, 128, 10, weight_norm=True)
        ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike
    
    def grad_flow(self, path):
        # helps monitor the gradient flow
        grad = [b.synapse.grad_norm for b in self.blocks if hasattr(b, 'synapse')]

        plt.figure()
        plt.semilogy(grad)
        plt.savefig(path + 'gradFlow.png')
        plt.close()

        return grad

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

In [22]:
trained_folder = '../models/Trained'
os.makedirs(trained_folder, exist_ok=True)

# device = torch.device('cpu')
device = torch.device('cuda') 

net = CNNSlayer().to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction='sum').to(device)
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)

In [12]:
from shared.system_monitor import SystemMonitor
monitor = SystemMonitor(sample_interval=1.0)

monitor.start()

epochs = 10
patience = 3
best_val_acc = 0.0
counter = 0

for epoch in range(epochs):
    # Training loop
    for i, (input, label) in enumerate(train_dataloader):
        output = assistant.train(input, label)
        if i % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {i+1}/{len(train_dataloader)}", end='\r')
    print(f'\r[Epoch {epoch+1}/{epochs}] {stats}', end='')

    # Validation loop
    val_correct = 0
    val_total = 0
    for i, (input, label) in enumerate(val_dataloader):
        output = assistant.test(input, label)
        pred = output.argmax(dim=1)
        val_correct += (pred == label).sum().item()
        val_total += label.size(0)
    val_acc = 100 * val_correct / val_total if val_total > 0 else 0
    print(f" | Validation Accuracy: {val_acc:.2f}%")

    # Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter = 0
        torch.save(net.state_dict(), trained_folder + '/network.pt')
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    stats.update()
    stats.save(trained_folder + '/')
    net.grad_flow(trained_folder + '/')

monitor.stop()

System monitor started.


TypeError: can't convert np.ndarray of type numpy.void. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.